<a href="https://colab.research.google.com/github/dr-song-summer-project/ChatBot_Preprocessing/blob/main/refining_Data/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
# from nltk.tokenize import word_tokenize  
# print(word_tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."))  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git   #띄어쓰기
!pip install git+https://github.com/ssut/py-hanspell.git          #맞춤법
!pip install soynlp                  #반복 제거
!pip install kss                     #문장 구분

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-l07gx__3
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-l07gx__3
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2255828 sha256=e1a9bbaec7e3f32271289498bbf78ec15149f1cf20a72676caa1cd8f81075254
  Stored in directory: /tmp/pip-ephem-wheel-cache-zt5bakbx/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
Successfully built pykospacing


  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-c651q4xq
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-c651q4xq
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4870 sha256=8b266ae7e62abc943e521ae9bba3d737063b5fbd8c8f6aca20dc8b49d328fb41
  Stored in directory: /tmp/pip-ephem-wheel-cache-a0a7c6_q/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell
     |████████████████████████████████| 416 kB 7.8 MB/s 
     |████████████████████████████████| 65 kB 3.1 MB/s 


In [ ]:
#기초적 전처리, html tag 제거, 불필요 언어 제거, Lowercasing, punctuation(문장부호) 제거
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&☺️' 
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 
def clean_punc(text, punct, mapping): 
  for p in mapping: 
    text = text.replace(p, mapping[p]) 

  for p in punct: 
    text = text.replace(p, f'{p}') 
    
  specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''} 
  for s in specials: 
    text = text.replace(s, specials[s]) 
    
  return text.strip()

# import re

# def clean_text(texts):   
#     corpus = []
#    for i in range(0, len(texts)) : 
#      review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation 
#      review = re.sub(r'\d+','', str(texts[i]))# remove number 
#      review = review.lower() #lower case 
#      review = re.sub(r'\s+', ' ', review) #remove extra space 
#      review = re.sub(r'<[^>]+>','',review) #remove Html tags 
#      review = re.sub(r'\s+', ' ', review) #remove spaces 
#      review = re.sub(r"^\s+", '', review) #remove space from start 
#      review = re.sub(r'\s+$', '', review) #remove space from the end 
#      corpus.append(review) 
#    return corpus

In [ ]:
from google.colab import files
files.upload()


Saving clustered_질문요약_800_n20_only.xlsx to clustered_질문요약_800_n20_only.xlsx


{'clustered_질문요약_800_n20_only.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00A7\x82\xcfn\x01\x00\x00\x04\x05\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
import pandas as pd
from openpyxl import Workbook
from pykospacing import Spacing 
from hanspell import spell_checker
from tqdm.notebook import tqdm
from soynlp.normalizer import *
import kss
import re

# ============================================
# --  텍스트 전처리
# ============================================
spacing = Spacing()
def spell_text(texts):
  sample_space = spacing(texts)
  # spelled_sent = spell_checker.check(sample_space)
  # sampel_space_spell = spelled_sent.checked
  return (sample_space)

def process_sentence(texts):
  chars = "<o:p/>br_ㅋ~ㅜㅠ앗ㅎ\\-&;\"굴림r"
  tmp = ''
  for sent_ in kss.split_sentences(texts):
    if(('안녕하세요' in sent_) or ('하이닥' in sent_) or ('감사' in sent_) or ('답변이 도움' in sent_) or ('문의에 도움' in sent_) or (len(sent_))>500) :
      continue
    if(('좋은 저녁 보내세요' in sent_) or ('답변을 참고'in sent_) or ('문의에 참고' in sent_) or ('죄송합니다.' in sent_) or ('인냥' in sent_)):
      continue
    if(('질문드려' in sent_) or ('안녕하세요' in sent_) or ('좋은 하루' in sent_) or ('그렇' in sent_) or ('채팅' in sent_)):
      continue
    if(('답변을 참고하시기 전에' in sent_) or ('1. 온라인 상에서의 답변은' in sent_) or ('2. 정확한 의학적 검진과 판단은' in sent_) or ('br' in sent_) or ('것이지요?' in sent_)):
      continue
    if(('잘 읽어보시고' in sent_) or ('답벼니 도움이 되었길' in sent_) or ('즐거운 하루' in sent_) or ('걱정 많으' in sent_) or('답변이 참고가' in sent_) ):
      continue
    if(('걱정 많이 되셨나' in sent_) or ('f nt' in sent_) or ('div' in sent_) or ('san' in sent_)):
      continue
    tmp = tmp + ' ' + (spell_text(sent_))
    tmp = clean_punc(tmp, punct, punct_mapping)
    #print(spell_text(sent_), end=' ')  
  for i in range(len(chars)):
    tmp = tmp.replace(chars[i],"")
  if(tmp == "" or tmp == "  "):
    return "내용이 없습니다."
  return tmp


In [ ]:
# ============================================
# --  파일 읽어오기
# ============================================
df = pd.read_excel('/content/clustered_질문요약_800_n20_only.xlsx')
df_np = pd.DataFrame.to_numpy(df)

processed_sentence_question = []
processed_sentence_answer = []

# tmp_q = df['질문'].tolist()
# tmp_a = df['답변'].tolist()

for i in tqdm(range(0, len(df))):
  try:
    tmp_q = process_sentence(df_np[i][2])
    tmp_a = process_sentence(df_np[i][4])
  except:
    print("error index : ", i )  
  processed_sentence_question.append(tmp_q)
  processed_sentence_answer.append(tmp_a)

# ============================================
# --  Excel 입력
# ============================================
wb = Workbook() 
sheet = wb.active 
sheet.title = '전처리된 문장' # 컬럼명 지정(헤더) 

sheet.cell(row=1, column=1, value='질문')
sheet.cell(row=1, column=2, value='답변')


# 시트 저장 
row_no = 2
for n, rows in tqdm(enumerate(processed_sentence_question)): 
  for seq, value in enumerate(rows): 
    sheet.cell(row=row_no+n, column=1, value=processed_sentence_question[n])
    sheet.cell(row=row_no+n, column=2, value=processed_sentence_answer[n])

wb.save('/content/result.xlsx')
wb.close()



In [ ]:
# ============================================
# --  Excel 입력
# ============================================
wb = Workbook() 
sheet = wb.active 
sheet.title = '전처리된 문장' # 컬럼명 지정(헤더) 

sheet.cell(row=1, column=1, value='질문')
sheet.cell(row=1, column=2, value='답변')


# 시트 저장 
row_no = 2
for n  in tqdm (range(len(processed_sentence_answer))): 
  sheet.cell(row=row_no+n, column=1, value=processed_sentence_question[n])
  sheet.cell(row=row_no+n, column=2, value=processed_sentence_answer[n])

wb.save('/content/sample_data_processed.xlsx')
wb.close()